In [ ]:
# Only run this if your are using google collab if not skip this cell
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def predict_behaviors_from_file(sensor_path, doginfo_path, model_path, label_encoder_path, scaler_seq_path, scaler_meta_path):
    """
    Predict dog behaviors from new sensor data using trained model.
    Returns a list of predicted labels.
    """
    import pandas as pd
    import numpy as np
    import joblib
    import seaborn as sns
    import matplotlib.pyplot as plt
    from tensorflow.keras.layers import LSTM
    from tensorflow.keras.models import load_model
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

    # === Custom LSTM handler to fix time_major error ===
    def lstm_without_time_major(*args, **kwargs):
        kwargs.pop('time_major', None)  # Remove problematic argument
        return LSTM(*args, **kwargs)

    # === Load new data ===
    sensor_cols = ['DogID', 'ANeck_x', 'ANeck_y', 'ANeck_z', 'GNeck_x', 'GNeck_y', 'GNeck_z']
    raw_df = pd.read_csv(sensor_path)

    has_label = 'Behavior_1' in raw_df.columns
    if has_label:
        sensor_cols.append('Behavior_1')

    sensor_data = raw_df[sensor_cols].dropna()

    if has_label:
        sensor_data = sensor_data[~sensor_data['Behavior_1'].isin(['<undefined>', 'Synchronization', 'Extra_Synchronization'])]

    doginfo = pd.read_excel(doginfo_path)

    # === Preprocess metadata ===
    doginfo['Weight'] /= doginfo['Weight'].max()
    doginfo['Age months'] /= doginfo['Age months'].max()
    doginfo = pd.get_dummies(doginfo, columns=['Breed', 'Gender', 'NeuteringStatus'])

    # Merge sensor + metadata
    full_df = pd.merge(sensor_data, doginfo, on='DogID', how='left')

    # === Create sequences ===
    def create_sequences(df, window_size=100, step=20):
        sequences, meta_features, labels = [], [], []
        feature_cols = ['ANeck_x', 'ANeck_y', 'ANeck_z', 'GNeck_x', 'GNeck_y', 'GNeck_z']
        static_cols = [col for col in df.columns if col not in feature_cols + ['DogID', 'Behavior_1']]

        for i in range(0, len(df) - window_size, step):
            window = df.iloc[i:i+window_size]
            if window['DogID'].nunique() > 1:
                continue

            label = window['Behavior_1'].iloc[window_size // 2] if has_label else None

            sequences.append(window[feature_cols].values)
            meta_features.append(window[static_cols].iloc[0].values)
            labels.append(label)

        return np.array(sequences), np.array(meta_features), np.array(labels)

    X_seq_new, X_meta_new, y_true_raw = create_sequences(full_df)

    if len(X_seq_new) == 0:
        print("❌ Not enough data or inconsistent DogID in sequences.")
        return []

    # === Scale inputs ===
    scaler_seq = joblib.load(scaler_seq_path)
    X_seq_flat = X_seq_new.reshape(-1, X_seq_new.shape[-1])
    X_seq_scaled = scaler_seq.transform(X_seq_flat).reshape(X_seq_new.shape)

    scaler_meta = joblib.load(scaler_meta_path)
    X_meta_scaled = scaler_meta.transform(X_meta_new)

    # === Load model with custom LSTM handler ===
    custom_objects = {'LSTM': lstm_without_time_major}
    try:
        model = load_model(model_path, custom_objects=custom_objects)
    except Exception as e:
        print(f"⚠️ Model loading failed: {str(e)}")
        print("Trying fallback without custom objects...")
        model = load_model(model_path)  # Fallback attempt

    label_encoder = joblib.load(label_encoder_path)

    # === Predict ===
    y_pred_probs = model.predict([X_seq_scaled, X_meta_scaled])
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_pred_labels = label_encoder.inverse_transform(y_pred)

    print("✅ Predictions completed.\n")

    # === Evaluation ===
    if has_label and y_true_raw[0] is not None:
        y_true_encoded = label_encoder.transform(y_true_raw)

        accuracy = accuracy_score(y_true_encoded, y_pred)
        print(f"✅ Accuracy: {accuracy:.4f}\n")

        print("🔍 Classification Report:")
        print(classification_report(
        y_true_encoded, 
        y_pred,
        labels=np.unique(y_true_encoded),  # Only use present classes
        target_names=label_encoder.classes_[np.unique(y_true_encoded)]  # Corresponding names
        ))

        cm = confusion_matrix(y_true_encoded, y_pred)
        plt.figure(figsize=(14, 10))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
                   xticklabels=label_encoder.classes_, 
                   yticklabels=label_encoder.classes_)
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix")
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    else:
        print("ℹ️ Skipped evaluation metrics because true labels were not provided.\n")

    return y_pred_labels.tolist(), y_true_raw.tolist() if has_label else None

In [ ]:
# Replace sensor_path with the new sensor-data path on your pc or drive
# Replace doginfo_path iwth the new doginfo data path on your pc or drive

predict_behaviors_from_file(
    sensor_path=sensor_path,
    doginfo_path=doginfo_path,
    model_path="dog_movement_model.h5",
    label_encoder_path="labelEncoder.pkl",
    scaler_seq_path = "scaler_seq.pkl",
    scaler_meta_path = "scaler_meta.pkl"
)
